#### Step 1: Import the necessary libraries

In [1]:
%pip install boto3 langchain python-dotenv

You should consider upgrading via the '/usr/local/bin/python3.10 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import boto3
from langchain.llms.bedrock import Bedrock
from langchain.embeddings import BedrockEmbeddings
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from dotenv import load_dotenv

#### Step 2: Load Environment Variables

In [4]:
load_dotenv()

aws_access_key_id = os.getenv("aws_access_key_id")
aws_secret_access_key = os.getenv("aws_secret_access_key")
region_name = os.getenv("region_name")

#### Step 3: Define Prompt Template

In [5]:
prompt_template = """
Human: Use the following pieces of context to provide a 
concise answer to the question at the end but use at least summarize with 
250 words with detailed explanations. If you don't know the answer, 
just say that you don't know, don't try to make up an answer.
<context>
{context}
</context>

Question: {question}

Assistant:"""

#### Step 4: Initialize Bedrock Client

In [6]:
bedrock = boto3.client(
    service_name="bedrock-runtime", 
    region_name=region_name,
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
)

#### Step 5: Get Embeddings Model

In [7]:
bedrock_embedding = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1", client=bedrock)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `BedrockEmbeddings` was deprecated in LangChain 0.2.11 and will be removed in 0.4.0. An updated version of the class exists in the langchain-aws package and should be used instead. To use it run `pip install -U langchain-aws` and import as `from langchain_aws import BedrockEmbeddings`.
  warn_deprecated(


#### Step 6: Function to Load Documents

In [8]:
def get_documents():
    loader = PyPDFDirectoryLoader("Data")  # Load all PDFs from the 'Data' directory
    documents = loader.load()
    text_spliter = RecursiveCharacterTextSplitter(
        chunk_size=1000, 
        chunk_overlap=500
    )
    docs = text_spliter.split_documents(documents)
    return docs

#### Step 7: Function to Create and Save Vector Store

In [9]:
def get_vector_store(docs):
    vectorstore_faiss = FAISS.from_documents(
        docs,
        bedrock_embedding
    )
    vectorstore_faiss.save_local("faiss_local")

#### Step 8: Function to Get Language Model

In [10]:
def get_llm():
    llm = Bedrock(model_id="mistral.mistral-7b-instruct-v0:2", client=bedrock)
    return llm

#### Step 9: Create Prompt Template Instance

In [11]:
PROMPT = PromptTemplate(
    template=prompt_template, 
    input_variables=["context", "question"]
)

#### Step 10: Function to Get LLM Response

In [12]:
def get_llm_response(llm, vectorstore_faiss, query):
    qa = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vectorstore_faiss.as_retriever(
            search_type="similarity", search_kwargs={"k": 3}
        ),
        return_source_documents=True,
        chain_type_kwargs={"prompt": PROMPT}
    )
    
    response = qa({"query": query})
    return response['result']

#### Step 11: Main Function to Run Chatbot

In [13]:
def main():
    print("Ayurveda Chatbot")
    print("Type 'exit' to quit the chat.")
    
    while True:
        user_question = input("Ask a question about Ayurveda: ")
        
        if user_question.lower() == 'exit':
            break
        
        faiss_index = FAISS.load_local("faiss_local", bedrock_embedding, allow_dangerous_deserialization=True) 
        llm = get_llm()
        
        response = get_llm_response(llm, faiss_index, user_question)
        print("\nAssistant:", response)